In [ ]:
import pdfplumber
import pandas as pd
import itertools

In [ ]:
pdf = pdfplumber.open("framtiden/Framtiden_arsredovisning_2020.pdf")

In [ ]:
pdf

In [ ]:
page = pdf.pages[116]

page = page.crop((56.238, 100, 555.278, 755)) #50, 560
im = page.to_image()

im

In [ ]:
table_settings = {
    "vertical_strategy": "text", 
    "horizontal_strategy": "lines",
    "explicit_vertical_lines": [],
    "explicit_horizontal_lines": [],
    "snap_tolerance": 3,
    "join_tolerance": 3,
    "edge_min_length": 3,
    "min_words_vertical": 3,
    "min_words_horizontal": 1,
    "keep_blank_chars": False,
    "text_tolerance": 3,
    "intersection_tolerance": 3,
}


im.reset().debug_tablefinder(table_settings)

In [ ]:
set([x[0] for x in page.find_tables(table_settings)[0].cells])

page.find_tables(table_settings)[0].bbox


In [ ]:
table_settings = {
    "vertical_strategy": "lines", 
    "horizontal_strategy": "lines",
    "explicit_vertical_lines": [56.238, 121.716, 150.626, 216.8005, 310, 335.985, 374.940, 398.572, 449.021, 495.298, 525.278, 555.278],
    "explicit_horizontal_lines": [],
    "snap_tolerance": 3,
    "join_tolerance": 3,
    "edge_min_length": 3,
    "min_words_vertical": 3,
    "min_words_horizontal": 1,
    "keep_blank_chars": False,
    "text_tolerance": 3,
    "text_x_tolerance": None,
    "text_y_tolerance": None,
    "intersection_tolerance": 3,
    "intersection_x_tolerance": None,
    "intersection_y_tolerance": None,
}


im.reset().debug_tablefinder(table_settings)

In [ ]:
page.extract_table(table_settings)

In [ ]:
#pages = pdf.pages[115:139]

#pages[0].to_image()

offset = 14

im = None
for p_number in range(115, 139 + 1):
    vertical_lines = [56.238 - 10,56.238, 121.716, 150.626, 216.8005, 310, 335.985, 374.940, 398.572, 449.021, 495.298, 525.278, 555, 555.278 + 10]
    p_crop = [56.238 - 10, 100, 555.278 + 10, 755 + 10]
    
    if p_number % 2 == 1:
        vertical_lines = [l_number - offset for l_number in vertical_lines]
        p_crop[0] -= offset
        p_crop[2] -= offset
         #tuple(x - offset for x in p_crop)
        
    p = pdf.pages[p_number]
    p = p.crop(p_crop) #50, 560
    
    im = p.to_image()
    
    table_settings = {
        "vertical_strategy": "lines", 
        "horizontal_strategy": "lines",
        "explicit_vertical_lines": vertical_lines,
        "explicit_horizontal_lines": [],
        "snap_tolerance": 3,
        "join_tolerance": 3,
        "edge_min_length": 3,
        "min_words_vertical": 3,
        "min_words_horizontal": 1,
        "keep_blank_chars": False,
        "text_tolerance": 3,
        "text_x_tolerance": None,
        "text_y_tolerance": None,
        "intersection_tolerance": 3,
        "intersection_x_tolerance": None,
        "intersection_y_tolerance": None,
    }
    
    if p_number == 116:
        break
        

im.reset().debug_tablefinder(table_settings)

In [ ]:
p.extract_table(table_settings)

In [ ]:
p.find_tables(table_settings)[0].rows[0].__dict__

In [ ]:
offset = 14

tables = []

#im = None
for p_number in range(115, 138 + 1):
    print(p_number)
    
    vertical_lines = [56.238 - 10,56.238, 121.716, 150.626, 216.8005, 310, 335.985, 374.940, 398.572, 449.021, 495.298, 525.278, 555, 555.278 + 10]
    p_crop = [56.238 - 10, 100, 555.278 + 10, 755 + 10]
    
    if p_number % 2 == 1:
        vertical_lines = [l_number - offset for l_number in vertical_lines]
        p_crop[0] -= offset
        p_crop[2] -= offset
         #tuple(x - offset for x in p_crop)
        
    p = pdf.pages[p_number]
    p = p.crop(p_crop) #50, 560
    
    #im = p.to_image()
    
    table_settings = {
        "vertical_strategy": "lines", 
        "horizontal_strategy": "lines",
        "explicit_vertical_lines": vertical_lines,
        "explicit_horizontal_lines": [],
        "snap_tolerance": 3,
        "join_tolerance": 3,
        "edge_min_length": 3,
        "min_words_vertical": 3,
        "min_words_horizontal": 1,
        "keep_blank_chars": False,
        "text_tolerance": 3,
        "text_x_tolerance": None,
        "text_y_tolerance": None,
        "intersection_tolerance": 3,
        "intersection_x_tolerance": None,
        "intersection_y_tolerance": None,
    }
    
    tables.append(p.extract_table(table_settings))
    
    #if p_number == 116:
    #    break
        

#im.reset().debug_tablefinder(table_settings)


tables

In [ ]:
pd.DataFrame(tables[0])

In [ ]:
first_line = ['',
  'Stadsdel/\nPrimär–\nområden',
  'Fastig-\nhets–\nägare 1)',
  'Fastighets–\nbeteckning',
  'Besöksadress V',
  'ärde år',
  'Bostads–\nyta, kvm',
  'Lokal–\nyta, \nkvm',
  'Antal \nbostäder',
  'Taxerings–\nvärde, tkr',
  'Hyres-\nvärde, \ntkr 2)',
  'Bostads-\nhyra,\nkr/kvm 3)',
  '']

for table in tables:
    assert table[0] == first_line
    
    for row in table:
        assert (row[0] == '' or row[0] is None) and (row[-1] == '' or row[-1] is None)

In [ ]:
def parse_table(raw_table):
    table = []
    
    assert raw_table[0] == first_line
    
    rows = []
    
    for raw_row in raw_table[1:]:
        row = raw_row[1:-1]
        
        if set(row) == {''}: # empty row
            continue
        
        if len(row[0]) > 0 and set(row[1:]) == {''}: # område title
            continue
        
        if len(row[0]) > 0 and set(row[1:5]) == {''}: # område sum
            continue
            
        if row[0].startswith('SUMMA'):
            continue
        
        #print(row)
        
        rows.append(row)
    
    return rows

rows = list(itertools.chain.from_iterable([parse_table(raw_table) for raw_table in tables]))

df = pd.DataFrame(rows, columns=['Stadsdel/Primärområden', 'Fastighetsägare', 'Fastighetsbeteckning', 'Besöksadress', 'Värde år', 'Bostads–yta, kvm', 'Lokal–yta, kvm', 'Antal bostäder', 'Taxerings–värde, tkr', 'Hyres-värde, tkr', 'Bostadshyra, kr/kvm'])
pd.set_option("display.max_rows", None, "display.max_columns", None)

df